In [1]:
import json
import geopandas as gpd
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine

In [2]:
# Load database configuration from file
with open(r'D:\iMMAP\code\db_config\hsdc_local_db_config.json', 'r') as f:
    config = json.load(f)

# Create database URL with credentials
db_url = f"postgresql://{config['username']}:{config['password']}@{config['host']}:{config['port']}/{config['database']}"

# Connect to the database
con = create_engine(db_url)

In [11]:
# Define projection
repro_crs = 'epsg:4326'

# Load data and reproject
av = gpd.GeoDataFrame.from_postgis('SELECT * from afg_avsa', con, 'wkb_geometry').to_crs(repro_crs)

In [24]:
# Cleaning table
av_drop = av.drop(columns=['dist_code', 'dist_na_en', 'prov_na_en', 'prov_code', 'vuid'])
av_drop['avalanche_pop'] = np.NaN
av_drop['area_buildings'] = np.NaN

,ogc_fid,wkb_geometry,avalanche_cat,avalanche_id,avalanche_zone,avalanche_area,avalanche_lenght_m,basin_id,source,sum_area_sqm,avalanche_pop,area_buildings,shape_length,shape_area,basinmember_id
0,768381,"MULTIPOLYGON (((70.71088 37.09350, 70.71086 37...",Moderate,314354,Chute and run-out area,276501.0,723.0,2.023402e+09,E. Hagen. iMMAP 2015,188279,NaN,NaN,0.023299,1.908724e-05,5064
1,768720,"MULTIPOLYGON (((71.22247 37.09170, 71.22250 37...",Moderate,314617,Trigger area,NaN,813.0,2.023402e+09,E. Hagen. iMMAP 2015,3651,NaN,NaN,0.003176,3.700156e-07,5068
2,768382,"MULTIPOLYGON (((70.71199 37.09384, 70.71202 37...",Moderate,314354,Trigger area,NaN,723.0,2.023402e+09,E. Hagen. iMMAP 2015,86825,NaN,NaN,0.015763,8.801988e-06,5064
3,769286,"MULTIPOLYGON (((70.74028 37.10278, 70.74028 37...",Moderate,315008,Trigger area,NaN,327.0,2.023402e+09,E. Hagen. iMMAP 2015,10665,NaN,NaN,0.004491,1.081289e-06,5064
4,768383,"MULTIPOLYGON (((70.84938 37.08758, 70.84951 37...",Moderate,314359,Chute and run-out area,63579.0,477.0,2.023402e+09,E. Hagen. iMMAP 2015,55085,NaN,NaN,0.015613,5.583398e-06,5066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
845525,768371,"MULTIPOLYGON (((70.88045 37.08697, 70.88048 37...",Moderate,314346,Chute and run-out area,131595.0,708.0,2.023402e+09,E. Hagen. iMMAP 2015,90268,NaN,NaN,0.018111,9.148943e-06,5066
845526,768372,"MULTIPOLYGON (((70.88268 37.08340, 70.88268 37...",Moderate,314346,Trigger area,NaN,708.0,2.023402e+09,E. Hagen. iMMAP 2015,42381,NaN,NaN,0.009058,4.295541e-06,5066
845527,768373,"MULTIPOLYGON (((70.23647 37.09702, 70.23647 37...",Moderate,314350,Chute and run-out area,51260.0,333.0,2.023402e+09,E. Hagen. iMMAP 2015,44403,NaN,NaN,0.009405,4.502934e-06,5184
845528,768374,"MULTIPOLYGON (((70.23647 37.09702, 70.23641 37...",Moderate,314350,Trigger area,NaN,333.0,2.023402e+09,E. Hagen. iMMAP 2015,6854,NaN,NaN,0.003869,6.950714e-07,5184


In [25]:
av_drop.to_postgis("afg_avsa_clean", con)

In [17]:
# Dissolving by province
av_dissolved = av.dissolve(by='avalanche_cat').explode()

av_dissolved
av_dissolved_clean = av_dissolved.set_index('ogc_fid')[['wkb_geometry', 'avalanche_cat']]

C:\Users\VMO\AppData\Local\Temp\ipykernel_11632\2930064340.py:2: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  av_dissolved = av.dissolve(by='avalanche_cat').explode()


,avalanche_id,avalanche_zone,avalanche_area,avalanche_lenght_m,dist_code,dist_na_en,prov_na_en,prov_code,basin_id,vuid,source,sum_area_sqm,avalanche_pop,area_buildings,shape_length,shape_area,basinmember_id,wkb_geometry
ogc_fid,,,,,,,,,,,,,,,,,,
737352,308721,Trigger area,8480.0,730,1521,Jorm,Badakhshan,15,2.023402e+09,JOM-009,E. Hagen. iMMAP 2015,20674,110.0,16.0,0.007326,2.091209e-06,5113,"POLYGON ((69.86896 36.96561, 69.86899 36.96556..."
737352,308721,Trigger area,8480.0,730,1521,Jorm,Badakhshan,15,2.023402e+09,JOM-009,E. Hagen. iMMAP 2015,20674,110.0,16.0,0.007326,2.091209e-06,5113,"POLYGON ((69.93032 36.96728, 69.93030 36.96735..."
737352,308721,Trigger area,8480.0,730,1521,Jorm,Badakhshan,15,2.023402e+09,JOM-009,E. Hagen. iMMAP 2015,20674,110.0,16.0,0.007326,2.091209e-06,5113,"POLYGON ((69.92114 36.96883, 69.92110 36.96875..."
737352,308721,Trigger area,8480.0,730,1521,Jorm,Badakhshan,15,2.023402e+09,JOM-009,E. Hagen. iMMAP 2015,20674,110.0,16.0,0.007326,2.091209e-06,5113,"POLYGON ((69.91351 36.97963, 69.91363 36.97955..."
737352,308721,Trigger area,8480.0,730,1521,Jorm,Badakhshan,15,2.023402e+09,JOM-009,E. Hagen. iMMAP 2015,20674,110.0,16.0,0.007326,2.091209e-06,5113,"POLYGON ((69.94776 36.98111, 69.94786 36.98116..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
745993,302197,Chute and run-out area,703756.0,1587,1522,Warduj,Badakhshan,15,2.023402e+09,WAD-010,E. Hagen. iMMAP 2015,4954,NaN,NaN,0.003087,4.996367e-07,5077,"POLYGON ((71.12409 36.71079, 71.12403 36.71081..."
745993,302197,Chute and run-out area,703756.0,1587,1522,Warduj,Badakhshan,15,2.023402e+09,WAD-010,E. Hagen. iMMAP 2015,4954,NaN,NaN,0.003087,4.996367e-07,5077,"POLYGON ((70.77356 36.72475, 70.77356 36.72472..."
745993,302197,Chute and run-out area,703756.0,1587,1522,Warduj,Badakhshan,15,2.023402e+09,WAD-010,E. Hagen. iMMAP 2015,4954,NaN,NaN,0.003087,4.996367e-07,5077,"POLYGON ((71.09780 36.72547, 71.09770 36.72552..."


In [ ]:
av_dissolved_clean

In [ ]:
# Save to PostGIS
av_dissolved_clean.to_postgis("afg_avsa_dissolve", con)